In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

# # Set the global seed value
# global_seed = random.randint(0, 1000)
# np.random.seed(global_seed)
# random.seed(global_seed)

class missile_interception_3d(gym.Env):
    def __init__(self):
        # 1. Define Action Space (The Joystick: Left/Right, Up/Down)
        self.action_space = gym.spaces.Box(low=-1.0, high=1.0, shape=(2, ), dtype=np.float32)
        
        # 2. Define Observation Space (The Eyes: Rel Pos, Rel Vel, Curr Accel, T_go, Fuel, Altitude, Speed)
        # Size = 3 (Rel Pos) + 3 (Rel Vel) + 2 (Curr Accel) + 1 (T_go) + 1 (Fuel) + 1 (Altitude) + 1 (Speed) = 12
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(12,), dtype=np.float32)

        self.np_random = np.random.RandomState()
        
        # 3. Time Settings
        self.dt_act = 0.1             # AI decides every 0.1 seconds
        self.n_substeps = 10          # Physics runs 10x faster
        self.dt_sim = self.dt_act / self.n_substeps # 0.01 seconds
        self.t_max = 650.0            # Maximum simulation time (seconds)

        # 4. Physical Limits - UPDATED FOR THRUST + DRAG
        self.g = 9.81        # Gravity
        
        # Rocket Motor
        self.thrust_magnitude = 400.0  # 40G acceleration (m/s²)
        self.burn_time = 8.0           # 8 seconds of solid fuel
        
        # Aerodynamics (Altitude-Dependent Drag)
        self.rho0 = 1.225              # Sea level air density (kg/m³)
        self.scale_height = 8500.0     # Atmosphere scale height (m)
        self.Cd = 0.5                  # Drag coefficient (supersonic missile)
        self.area = 0.03               # Cross-sectional area (m²) - ~20cm diameter
        self.mass = 150.0              # Missile mass (kg)
        
        # Maneuvering Limits
        self.a_max = 294.0             # 30G turn capability (m/s²)
        self.da_max = 2500.0           # Jerk limit (m/s³)
        self.tau = 0.05                # Airframe lag (seconds)
        
        # Other limits
        self.collision_radius = 10.0   # Hit radius (meters)
        self.max_distance = 4_000_000.0  # Maximum distance before truncation (4,000 km)
        
        # Dynamic Pressure Limits
        self.q_max = 80_000.0  # Maximum dynamic pressure (Pa) - structural limit

        self.targetbox_x_min = -15000
        self.targetbox_x_max = 15000
        self.targetbox_y_min = -15000
        self.targetbox_y_max = 15000

    def generate_enemy_missile(self):
        range_min, range_max = 70000, 1000000
        self.attack_target_x = self.np_random.uniform(self.targetbox_x_min, self.targetbox_x_max)
        self.attack_target_y = self.np_random.uniform(self.targetbox_y_min, self.targetbox_y_max)

        self.enemy_launch_angle = self.np_random.uniform(0, 2 * np.pi)
        self.enemy_theta = self.np_random.uniform(0.523599, 1.0472)
        lower_limit = np.sqrt((range_min * self.g) / np.sin(2 * self.enemy_theta))
        upper_limit = np.sqrt((range_max * self.g) / np.sin(2 * self.enemy_theta))

        self.enemy_initial_velocity = self.np_random.uniform(lower_limit, upper_limit)
        range = self.enemy_initial_velocity * np.cos(self.enemy_theta) * (2 * self.enemy_initial_velocity * np.sin(self.enemy_theta) / self.g)

        self.enemy_launch_x = self.attack_target_x + range * np.cos(self.enemy_launch_angle)
        self.enemy_launch_y = self.attack_target_y + range * np.sin(self.enemy_launch_angle)
        self.enemy_z = 0

        self.enemy_x = self.enemy_launch_x
        self.enemy_y = self.enemy_launch_y
        self.enemy_pos = np.array([self.enemy_x, self.enemy_y, self.enemy_z], dtype=np.float32)

        self.enemy_azimuth = (self.enemy_launch_angle + np.pi) % (2 * np.pi)

    def generate_defense_missile(self):
        self.defense_launch_x = self.np_random.uniform(self.targetbox_x_min, self.targetbox_x_max)
        self.defense_launch_y = self.np_random.uniform(self.targetbox_y_min, self.targetbox_y_max)
        # print("self.enemy_launch_angle: ", self.enemy_launch_angle)
        self.defense_azimuth = self.np_random.uniform((self.enemy_launch_angle - 0.785398), (self.enemy_launch_angle + 0.785398))
        self.defense_theta = 0.785398
        # CHANGED: Start at rail speed, not hypersonic
        self.defense_initial_velocity = 50.0  # Launch rail exit speed
        self.defense_x = self.defense_launch_x
        self.defense_y = self.defense_launch_y
        self.defense_z = 0
        self.defense_pos = np.array([self.defense_x, self.defense_y, self.defense_z], dtype=np.float32)

        self.defense_ax = 0
        self.defense_ay = 0
        self.defense_az = 0
    
    def _rate_limit_norm(self, a_cmd, a_prev, da_max, dt):
        """Norm-based rate limiter: limits the magnitude of acceleration change."""
        delta = a_cmd - a_prev
        max_delta = da_max * dt
        dnorm = float(np.linalg.norm(delta))
        if dnorm <= max_delta or dnorm < 1e-9:
            return a_cmd
        return a_prev + delta * (max_delta / dnorm)
    
    def _segment_sphere_intersect(self, r0, r1, r_hit):
        """Check if line segment from r0 to r1 intersects sphere of radius r_hit.
        Uses segment-sphere intersection: checks if closest point on segment to origin is within radius.
        """
        # Segment: r(s) = r0 + s*(r1 - r0), s in [0, 1]
        dr = r1 - r0
        dr_norm_sq = float(np.dot(dr, dr))
        
        # If segment is degenerate (zero length), check if r0 is within sphere
        if dr_norm_sq < 1e-12:
            return float(np.dot(r0, r0)) <= r_hit * r_hit
        
        # Find parameter s* where closest point on segment to origin occurs
        # s* = -dot(r0, dr) / dot(dr, dr), clamped to [0, 1]
        s_star = -float(np.dot(r0, dr)) / dr_norm_sq
        s_star = max(0.0, min(1.0, s_star))
        
        # Closest point on segment
        r_closest = r0 + s_star * dr
        
        # Check if within sphere
        return float(np.dot(r_closest, r_closest)) <= r_hit * r_hit
    
    def _get_obs(self):
        # 1. Relative Position & Velocity
        rel_pos = self.enemy_pos - self.defense_pos
        rel_vel = self.enemy_vel - self.defense_vel
        
        dist = np.linalg.norm(rel_pos)
        
        # 2. Closing Velocity (Scalar)
        # Dot product of position and velocity divided by distance
        # V_c is positive if closing, negative if opening
        v_close = -np.dot(rel_pos, rel_vel) / (dist + 1e-6)
        
        # 3. Time to Go (Estimated)
        t_go = dist / (v_close + 1e-6) if v_close > 0 else 0.0
        
        # 4. Current Acceleration in lateral basis (consistent with action space)
        # Compute the same basis as in step()
        right, up = self._compute_lateral_basis(self.defense_vel)
        
        # Project actual acceleration onto right/up basis
        a_lat = np.array([
            np.dot(self.a_actual, right) / self.a_max,
            np.dot(self.a_actual, up) / self.a_max,
        ], dtype=np.float32)
        
        # Add fuel remaining (1.0 = full, 0.0 = empty)
        fuel_remaining = max(0.0, (self.burn_time - self.t) / self.burn_time)
        
        # Add current altitude (normalized by 20km) - helps agent learn lofting faster
        altitude_normalized = self.defense_pos[2] / 20000.0
        
        # Add velocity magnitude (normalized by max expected speed ~3000 m/s) - helps energy management
        speed_normalized = np.linalg.norm(self.defense_vel) / 3000.0
        
        obs = np.concatenate([
            rel_pos / 10000.0, # Scale down for Neural Net stability
            rel_vel / 5000.0,
            a_lat,             # Lateral acceleration in action basis
            [t_go / 100.0],    # Scale time
            [fuel_remaining],  # Fuel state
            [altitude_normalized],  # Altitude (helps learn lofting)
            [speed_normalized]      # Speed magnitude (helps energy management)
        ], axis=0).astype(np.float32)
        
        return obs

    def _compute_lateral_basis(self, velocity):
        """Compute right/up basis vectors perpendicular to velocity (forward direction)."""
        speed = np.linalg.norm(velocity)
        # Clamp minimum speed to prevent numerical issues
        if speed < 1.0:
            forward = np.array([1.0, 0.0, 0.0], dtype=np.float32)
        else:
            forward = velocity / speed
        
        # Handle "Straight Up" edge case - use reference axis guaranteed nonparallel
        if abs(forward[2]) > 0.99:
            # Use a reference axis that's not parallel to forward
            ref_axis = np.array([1.0, 0.0, 0.0], dtype=np.float32)
            if abs(np.dot(forward, ref_axis)) > 0.9:
                ref_axis = np.array([0.0, 1.0, 0.0], dtype=np.float32)
            right_raw = np.cross(forward, ref_axis)
            right = right_raw / (np.linalg.norm(right_raw) + 1e-6)
        else:
            world_up = np.array([0.0, 0.0, 1.0], dtype=np.float32)
            right_raw = np.cross(forward, world_up)
            right = right_raw / (np.linalg.norm(right_raw) + 1e-6)
        
        # Normalize up vector
        up_raw = np.cross(right, forward)
        up = up_raw / (np.linalg.norm(up_raw) + 1e-6)
        
        return right, up

    def step(self, action):
        # 0. Guard: don't step if already done
        if getattr(self, "done", False):
            obs = self._get_obs()
            return obs, 0.0, True, False, {"event": "called_step_after_done", "dist": self.relative_distances[-1] if self.relative_distances else float('inf'), "t": self.t}
        
        # 1. Sanitize Action
        action = np.clip(action, -1.0, 1.0).astype(np.float32)
        mag = np.linalg.norm(action)
        if mag > 1.0:
            action = action / mag

        # Store distance before step for dense reward
        dist_before = float(np.linalg.norm(self.enemy_pos - self.defense_pos))

        # 2. THE SUBSTEP LOOP (Physics Engine)
        # Option B: Recompute basis each substep for physically consistent lateral steering
        terminated = False
        truncated = False
        event = "running"
        
        for _ in range(self.n_substeps):
            dt = self.dt_sim
            
            # Store old positions for segment-sphere collision detection
            enemy_pos_old = self.enemy_pos.copy()
            defense_pos_old = self.defense_pos.copy()
            
            # A. Recompute lateral basis from current velocity (Option B)
            # This ensures acceleration is always perpendicular to current velocity direction
            right, up = self._compute_lateral_basis(self.defense_vel)
            
            # B. Calculate Target Acceleration in World Frame (in current lateral basis)
            # Compute dynamic pressure for authority scaling (done after rho/speed calculation)
            # We'll compute q here, but need to get it from the physics section
            # For now, compute a preliminary q estimate for authority scaling
            z_curr_prelim = max(0.0, self.defense_pos[2])
            rho_prelim = self.rho0 * np.exp(-z_curr_prelim / self.scale_height)
            speed_prelim = np.linalg.norm(self.defense_vel)
            q_prelim = 0.5 * rho_prelim * (speed_prelim**2)
            
            # Scale lateral authority down when q is high (soft limit)
            # authority = 1.0 at low q, goes to ~0 as q >> q_max
            authority = 1.0 / (1.0 + (q_prelim / self.q_max)**2)
            a_max_effective = self.a_max * authority
            
            # action[0] = Horizontal Turn (Right/Left)
            # action[1] = Vertical Turn (Up/Down)
            a_target_world = (action[0] * a_max_effective * right) + (action[1] * a_max_effective * up)
            
            # C. Rate Limiter (Muscle Limit) - Norm-based
            self.a_cmd_prev = self._rate_limit_norm(a_target_world, self.a_cmd_prev, self.da_max, dt)
            
            # D. Lag (Inertia)
            # The fins move, but the body takes time to react (Low-pass filter)
            self.a_actual += (self.a_cmd_prev - self.a_actual) * (dt / self.tau)
            
            # ============================================================
            # NEW PHYSICS: THRUST + ALTITUDE-DEPENDENT DRAG
            # ============================================================
            
            # 1. ATMOSPHERE - Air density decreases exponentially with altitude
            z_curr = max(0.0, self.defense_pos[2])  # Clamp to ground level
            rho = self.rho0 * np.exp(-z_curr / self.scale_height)
            
            # 2. DYNAMIC PRESSURE - q = 0.5 * rho * v²
            speed = np.linalg.norm(self.defense_vel)
            q = 0.5 * rho * (speed**2)
            
            # 3. VELOCITY DIRECTION - Use initial forward direction if speed is too low
            if speed > 20.0:  # Threshold for using actual velocity direction
                velocity_dir = self.defense_vel / speed
            else:
                velocity_dir = self.initial_forward_dir
            
            # 4. DRAG FORCE - F_drag = 0.5 * rho * v² * Cd * A
            force_drag_mag = 0.5 * rho * (speed**2) * self.Cd * self.area
            force_drag = -force_drag_mag * velocity_dir
            
            # 5. THRUST FORCE - Only active during burn time
            if self.t < self.burn_time:
                # Thrust always points in velocity direction (gravity-turn guidance)
                force_thrust_mag = self.thrust_magnitude * self.mass
                force_thrust = force_thrust_mag * velocity_dir
            else:
                force_thrust = np.array([0.0, 0.0, 0.0], dtype=np.float32)
            
            # 6. GUIDANCE FORCE - AI control (lateral acceleration)
            force_guidance = self.a_actual * self.mass
            
            # 7. GRAVITY FORCE
            force_gravity = np.array([0.0, 0.0, -self.g * self.mass], dtype=np.float32)
            
            # 8. TOTAL ACCELERATION (Newton's Second Law: F = ma)
            total_force = force_thrust + force_drag + force_guidance + force_gravity
            total_accel = total_force / self.mass
            
            # 9. INTEGRATE DEFENSE MISSILE
            self.defense_vel += total_accel * dt
            self.defense_pos += self.defense_vel * dt
            # Keep individual variables in sync
            self.defense_x, self.defense_y, self.defense_z = self.defense_pos
            
            # ============================================================
            # ENEMY MISSILE (unchanged - still ballistic)
            # ============================================================
            accel_gravity = np.array([0.0, 0.0, -self.g], dtype=np.float32)
            self.enemy_vel += accel_gravity * dt
            self.enemy_pos += self.enemy_vel * dt
            # Keep individual variables in sync
            self.enemy_x, self.enemy_y, self.enemy_z = self.enemy_pos
            
            # Update time
            self.t += dt
            
            # H. Check Collision using segment-sphere intersection
            r0 = enemy_pos_old - defense_pos_old
            r1 = self.enemy_pos - self.defense_pos
            if self._segment_sphere_intersect(r0, r1, self.collision_radius):
                self.success = True
                terminated = True
                self.done = True
                event = "hit"
                break
            
            # I. Check if missiles have diverged too far
            dist = float(np.linalg.norm(self.enemy_pos - self.defense_pos))
            if dist > self.max_distance:
                truncated = True
                self.done = True
                event = "diverged"
                break
                
            # J. Check Ground Collision
            if self.defense_pos[2] < 0 or self.enemy_pos[2] < 0:
                terminated = True
                self.done = True
                event = "ground"
                break
            
            # K. Check Timeout
            if self.t >= self.t_max:
                truncated = True
                self.done = True
                event = "timeout"
                break

        # 5. Logging & History
        self.enemy_path.append(self.enemy_pos.copy())
        self.defense_path.append(self.defense_pos.copy())
        self.relative_distances.append(float(np.linalg.norm(self.enemy_pos - self.defense_pos)))
        self.times.append(self.t)
        
        # 6. Returns
        obs = self._get_obs()
        
        # Reward: Dense shaping + sparse success
        dist_after = float(np.linalg.norm(self.enemy_pos - self.defense_pos))
        reward = -1.0 * self.dt_act  # Time-consistent step penalty
        # Dense reward: normalize distance change by characteristic scale (10km)
        # This keeps reward stable even when distance changes by large amounts
        reward += (dist_before - dist_after) / 10000.0
        if self.success:
            reward += 1000.0
        
        info = {
            "dist": self.relative_distances[-1] if self.relative_distances else float('inf'),
            "event": event,
            "t": self.t
        }
        
        return obs, reward, terminated, truncated, info

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        if seed is not None:
            self.np_random = np.random.RandomState(seed)
        
        self.done = False
        self.success = False
        self.t = 0.0
        
        self.generate_enemy_missile()
        self.generate_defense_missile()
        
        # Define initial forward direction (for low-speed fallback)
        self.initial_forward_dir = np.array([
            np.cos(self.defense_azimuth) * np.cos(self.defense_theta),
            np.sin(self.defense_azimuth) * np.cos(self.defense_theta),
            np.sin(self.defense_theta)
        ], dtype=np.float32)
        self.initial_forward_dir /= (np.linalg.norm(self.initial_forward_dir) + 1e-9)
        
        # Initialize velocities
        self.defense_vel = np.array([
            self.defense_initial_velocity * np.cos(self.defense_azimuth) * np.cos(self.defense_theta),
            self.defense_initial_velocity * np.sin(self.defense_azimuth) * np.cos(self.defense_theta),
            self.defense_initial_velocity * np.sin(self.defense_theta)
        ], dtype=np.float32)
        
        self.enemy_vel = np.array([
            self.enemy_initial_velocity * np.cos(self.enemy_azimuth) * np.cos(self.enemy_theta),
            self.enemy_initial_velocity * np.sin(self.enemy_azimuth) * np.cos(self.enemy_theta),
            self.enemy_initial_velocity * np.sin(self.enemy_theta)
        ], dtype=np.float32)
        
        # Initialize acceleration state
        self.a_actual = np.zeros(3, dtype=np.float32)
        self.a_cmd_prev = np.zeros(3, dtype=np.float32)
        
        # Update positions to match pos arrays
        self.defense_pos = np.array([self.defense_x, self.defense_y, self.defense_z], dtype=np.float32)
        self.enemy_pos = np.array([self.enemy_x, self.enemy_y, self.enemy_z], dtype=np.float32)
        
        # Initialize paths/distances with initial state
        self.enemy_path = [self.enemy_pos.copy()]
        self.defense_path = [self.defense_pos.copy()]
        self.relative_distances = [float(np.linalg.norm(self.enemy_pos - self.defense_pos))]
        self.times = [self.t]
        
        return self._get_obs(), {}

    def graph_initial(self):
        plt.figure()
        plt.plot(self.enemy_x, self.enemy_y, 'ro', label="Enemy missile")
        plt.plot(self.defense_x, self.defense_y, 'bo', label="Defense missile")
        plt.plot(self.attack_target_x, self.attack_target_y, 'go', label="Attack target")
        plt.xlabel('X coordinate (m)')
        plt.ylabel('Y coordinate (m)')
        plt.title('Missile interception')
        plt.legend()
        plt.axis('equal')  # Ensures 1:1 aspect ratio
        plt.show()

    def render(self):
        enemy_path_array = np.array(self.enemy_path)
        defense_path_array = np.array(self.defense_path)
        print("enemy_path_array: ", enemy_path_array)
        print("defense_path_array: ", defense_path_array)    
        xe, ye, ze = enemy_path_array[:, 0], enemy_path_array[:, 1], enemy_path_array[:, 2]

        xd, yd, zd = defense_path_array[:, 0], defense_path_array[:, 1], defense_path_array[:, 2]

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot(xe, ye, ze, label="Enemy missile trajectory", color='blue')
        ax.plot(xd, yd, zd, label="Defense missile trajectory", color='red')
        ax.scatter(xe[0], ye[0], ze[0], color='blue', s=50, label="Enemy Start")
        ax.scatter(xd[0], yd[0], zd[0], color='red', s=50, label="Defense Start")
        ax.set_xlabel('X coordinate (m)')
        ax.set_ylabel('Y coordinate (m)')
        ax.set_zlabel('Z coordinate (m)')
        ax.set_title('Missile Trajectory')
        ax.legend()
        plt.show()



In [4]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
from IPython.display import HTML, display

# ==========================================
# 1. RUN SINGLE DEBUG SIMULATION
# ==========================================
env = missile_interception_3d()
obs, _ = env.reset()
done = False
step_count = 0

print(f"{'TIME':<8} | {'ALTITUDE':<10} | {'SPEED':<10} | {'FUEL %':<8} | {'STATUS'}")
print("-" * 55)

while not done:
    # Action [0,0] = No steering. Pure physics test.
    # The missile should gravity turn naturally.
    obs, reward, terminated, truncated, info = env.step(np.array([0.0, 0.0], dtype=np.float32))
    done = terminated or truncated
    
    # Extract Physics Data for Debugging
    # Note: obs[8] is fuel, obs[11] is speed (normalized), but let's use raw env data for clarity
    speed = np.linalg.norm(env.defense_vel)
    alt = env.defense_pos[2]
    fuel_pct = max(0.0, (env.burn_time - env.t) / env.burn_time) * 100.0
    
    # Print telemetry every 1.0 seconds (approx 10 steps since dt_act=0.1)
    if step_count % 10 == 0:
        event_str = "BOOST" if fuel_pct > 0 else "COAST"
        if info.get('event') != 'running':
            event_str = info.get('event').upper()
            
        print(f"{env.t:<8.2f} | {alt:<10.1f} | {speed:<10.1f} | {fuel_pct:<8.1f} | {event_str}")

    step_count += 1

print("-" * 55)
print(f"FINAL RESULT: {info.get('event', 'unknown')} at t={env.t:.2f}s")
print(f"Defense Hit Ground? {env.defense_pos[2] < 0}")


# ==========================================
# 2. GENERATE ANIMATION
# ==========================================
def update_paths(num, xe, ye, ze, xd, yd, zd, lines, ax):
    # Update the data in the trajectory plots
    # We slice [:num] to show the path growing over time
    lines[0].set_data_3d(xe[:num], ye[:num], ze[:num])
    lines[1].set_data_3d(xd[:num], yd[:num], zd[:num])

    # Rotate the camera angle slightly every frame for 3D effect
    ax.view_init(elev=20, azim=-60 + (num * 0.2)) 
    return lines

def animate_trajectories(enemy_path, defense_path):
    enemy_path_array = np.array(enemy_path)
    defense_path_array = np.array(defense_path)

    # Extract coordinates
    xe, ye, ze = enemy_path_array[:, 0], enemy_path_array[:, 1], enemy_path_array[:, 2]
    xd, yd, zd = defense_path_array[:, 0], defense_path_array[:, 1], defense_path_array[:, 2]

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Calculate bounds to keep the plot centered
    all_x = np.concatenate((xe, xd))
    all_y = np.concatenate((ye, yd))
    all_z = np.concatenate((ze, zd))
    
    ax.set_xlim([np.min(all_x), np.max(all_x)])
    ax.set_ylim([np.min(all_y), np.max(all_y)])
    ax.set_zlim([0, np.max(all_z)]) # Z should start at 0 (ground)

    # Initialize lines
    line_enemy, = ax.plot([], [], [], 'b-', linewidth=2, label="Enemy (Ballistic)")
    line_defense, = ax.plot([], [], [], 'r-', linewidth=2, label="Defense (Interceptor)")
    
    # Plot start points
    ax.scatter(xe[0], ye[0], ze[0], color='blue', s=50, marker='o')
    ax.scatter(xd[0], yd[0], zd[0], color='red', s=50, marker='^')

    lines = [line_enemy, line_defense]

    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Altitude (m)')
    ax.set_title(f'Interception Simulation\nMax Speed: {np.max(np.linalg.norm(defense_path_array, axis=1)):.0f} m/s')
    ax.legend()

    # Determine frames (subsample if too long to save rendering time)
    total_steps = len(xe)
    step_size = max(1, total_steps // 200) # Limit to ~200 frames max
    frames = range(0, total_steps, step_size)

    # Create Animation
    ani = animation.FuncAnimation(
        fig, update_paths, len(frames), 
        fargs=(xe[::step_size], ye[::step_size], ze[::step_size], 
               xd[::step_size], yd[::step_size], zd[::step_size], 
               lines, ax),
        interval=30, blit=False)

    plt.close(fig) # Prevent static plot from showing up
    
    # Return HTML video
    return HTML(ani.to_html5_video())

# Run the animation function
# Note: This might take 10-20 seconds to render depending on simulation length
print("Rendering Animation...")
animate_trajectories(env.enemy_path, env.defense_path)

TIME     | ALTITUDE   | SPEED      | FUEL %   | STATUS
-------------------------------------------------------
0.10     | 5.0        | 89.3       | 98.8     | BOOST
1.10     | 200.7      | 477.0      | 86.2     | BOOST
2.10     | 648.2      | 844.3      | 73.8     | BOOST
3.10     | 1326.4     | 1181.3     | 61.3     | BOOST
4.10     | 2212.2     | 1485.9     | 48.8     | BOOST
5.10     | 3283.9     | 1762.2     | 36.3     | BOOST
6.10     | 4524.0     | 2017.4     | 23.8     | BOOST
7.10     | 5920.6     | 2259.3     | 11.3     | BOOST
8.10     | 7465.6     | 2459.5     | 0.0      | COAST
9.10     | 9010.9     | 2320.3     | 0.0      | COAST
10.10    | 10471.8    | 2213.7     | 0.0      | COAST
11.10    | 11866.1    | 2129.7     | 0.0      | COAST
12.10    | 13206.3    | 2061.7     | 0.0      | COAST
13.10    | 14501.3    | 2005.8     | 0.0      | COAST
14.10    | 15757.9    | 1958.9     | 0.0      | COAST
15.10    | 16981.2    | 1919.1     | 0.0      | COAST
16.10    | 18175.5    | 1